In [ ]:
# @title Dependencies installation

# @markdown Run this code just once, after setting up the docker.

from time import time
start = time()

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pt0jsdnLrvDya7QMyQvh_qmEX2lifalz' -O requirements.txt
#!pip uninstall -y keras tensorflow tensorflow-probability absl-py astunparse flatbuffers gast google-pasta grpcio h5py keras keras-preprocessing libclang numpy opt-einsum protobuf setuptools six tensorboard tensorflow-io-gcs-filesystem termcolor tf-estimator-nightly typing-extensions wrapt
!pip install -r requirements.txt --no-deps

!pip install pyocclient

print(f"Installation took {int(time()-start)}s.")
print("Please click on 'Restart Session' button above.")

In [ ]:
!python --version

In [ ]:
# @title Information
# @markdown ##About

#@markdown With this colab, it is possible to train a single or multiple VST3-Models. </br>
#@markdown They can be used to synth or tone-shift input audio to an instrument that can be trained here </br>

#@markdown This colab was modified to be usable using the ressources of a local machine,
#@markdown that runs a colab-docker image. </br>
#@markdown For further information, see here:  [research.google.com](https://research.google.com/colaboratory/local-runtimes.html)


# @markdown ##Content

#@markdown - With the first cell it is possible to train one model
#@markdown - The second one allows the training of multiple models
#@markdown   - The run of the first cell (with *Run_Training* set to *False*) is still necessary
#@markdown - As for the visualization of the loss during the training, the last cell can be called

In [ ]:
# ------------------------------------------------------------------------------
#     Copyright 2022 Google LLC. All Rights Reserved.
#
#     Licensed under the Apache License, Version 2.0 (the "License");
#     you may not use this file except in compliance with the License.
#     You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0
#
#     Unless required by applicable law or agreed to in writing, software
#     distributed under the License is distributed on an "AS IS" BASIS,
#     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#     See the License for the specific language governing permissions and
#     limitations under the License.
# ------------------------------------------------------------------------------


# ------------------------------------------------------------------------------
# User Interface
# ------------------------------------------------------------------------------

# @title DDSP-VST Model Training
#@markdown #  Train your own DDSP-VST Model (docker version)
#@markdown 🎻🎺🎸🎵 [g.co/magenta/train-ddsp-vst](g.co/magenta/train-ddsp-vst)

#@markdown <br/>

#@markdown ## Instructions

#@markdown * Create a folder in Google Drive with your training audio (`.wav` or `.mp3`)

from keras.dtensor import dtensor_api as dtensor

Name = 'mymodel' #@param {type:"string"}
Name = Name.replace(' ', '_')

#@markdown <br/> Link to tubCloud dataset directory/archive with audio files. The link should be public. TODO
Dataset_Link = 'https://tubcloud.tu-berlin.de/s/3gmWPiK4xKRjXKd'  #@param {type:"string"}

#@markdown <br/>
#@markdown Use Cloud services for training? Otherwise, loads audio from browser, which is much slower.
Cloud_Service = "TUB-Cloud/Nextcloud" #@param ["Google Drive", "TUB-Cloud/Nextcloud", "None"]

Cloud_File_Name = 'wolves.wav.zip' #@param {type:"string"}

# Global declerations
CLOUD_NONE = 0
CLOUD_NEXTCLOUD = 1
CLOUD_GDRIVE = 2

Run_Training = True #@param {type:"boolean"}

#@markdown * Set Run_Training to **true**

#@markdown * Press the ▶️ button in the upper left!

#@markdown * Login to your Google account when asked

#@markdown *  Select your folder with the file chooser below when asked

#@markdown *  Wait (with this window open) for training to finish and download the model

#@markdown *  If something breaks, resume training by refreshing this page, press ▶️, and choose the same folder



#@markdown <br/>

#@markdown <br/>

#@markdown ## Data
#@markdown Custom models can train on as little as 10 minutes of audio (`.wav` or `.mp3`). You can get the best results from "monophonic" (only one note at a time) audio from a single recording session (same mic, same reverb). All of your data is private, used locally, and erased as soon as your colab session ends.

#@markdown We recommend using Google Drive to load data faster and save your model during training. Just create a folder on your drive with your audio files in it, and select the folder. If you don't use drive, you can still upload audio through the browser (slower) and download the final trained model.


#@markdown ## Training
#@markdown Training typically takes ~2-3 hours with free Colab, and less than an hour with ColabPro+. Free colab can sometimes disconnects before models finish training, but there are some unofficial [ways around this](https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting). If you do get disconnected, don't worry, just press play again and choose the same folder. The training will resume where it left off.




#@markdown ## Export

#@markdown After training, the colab should automatically export, zip, and download your model folder. To use, just unzip and drop the full folder in plugin model folder (Mac: `~/Documents/Magenta/DDSP/Models`, which you can also find from inside the plugin).

#@markdown If it doesn't automatically download, you can also find it in your training folder (`ddsp-training-{date-time}/{Name}`). Also, you'll likely see a bunch of warnings like `Value in checkpoint could not be found in the restored object`, don't worry that's normal :).


#@markdown <br/> <br/>
#@markdown ## Advanced Options

##@markdown <a href="https://colab.research.google.com/github/magenta/ddsp/blob/main/ddsp/colab/demos/Train_VST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#@markdown <br/>
#@markdown  Usually we will produce good results after training between 30k-50k steps, but your results may vary depending on your audio files/instrument. Too few steps will often have the model sound bland/generic, too many steps can often lead to more "sputtering" and big volume fluctuations

Training_Steps = 50000 #@param {type:"integer"}

Steps_Per_Save = 1000 #@param {type:"integer"}

#@markdown <br/>
#@markdown Restart the training each time at mulitple of this step (0=no restart).<br/>
#@markdown TODO: fix the generated .gin file (incorrect comment on # of taining steps)
Threshold = 1000 #@param {type:"integer", min:0, max:50000}

#@markdown At which training step should the restart using <i>Threshold</i> begin.
#@markdown Useful if a previous training got interrupted.
Threshold_Start = 0 #@param {type:"integer", min:0, max:50000}

#@markdown <br/>
#@markdown Ignore previous checkpoints in the folder and start a fresh run from step 0

Ignore_Previous = False #@param {type:"boolean"}

#@markdown <br/>
#@markdown Disconnect from session after the training is completed. <br/>
#@markdown Use it only when <b>Google_Drive</b> is enabled. The model-ZIP will be saved there. TODO REMOVE/REPLACE WITH SHUTDOWN
Disconnect_Finished = False #@param {type:"boolean"}


##@markdown ---
##@markdown <sub> This notebook sends anonymous usage data (i.e. training time) to Google Analytics to help improve/debug training. All audio and model information is private, and not sent or stored. For more information, see [Google's privacy policy](https://policies.google.com/privacy). </sub>




# ------------------------------------------------------------------------------
# Imports (not DDSP Dependent)
# ------------------------------------------------------------------------------
# Supress warnings that obscure output.
import warnings
warnings.filterwarnings("ignore")

import datetime
import glob
import os
import shutil
import time
import IPython
import json
import subprocess
import termcolor

from google.colab import drive, runtime
import tensorflow as tf

from ipyfilechooser import FileChooser

# ------------------------------------------------------------------------------
# Import DDSP
# ------------------------------------------------------------------------------
from ddsp.colab import colab_utils
globals()['colab_utils'] = colab_utils


# Global variables
export_path = ''


# ------------------------------------------------------------------------------
# Functions
# ------------------------------------------------------------------------------
def get_data_from_nextcloud(public_share_link, dataset_fn):
    import owncloud
    bn = os.path.basename(dataset_fn)
    bn = bn.lower()
    bn = bn.replace(" ", "_")

    try:
        oc = owncloud.Client.from_public_link(public_share_link)
    except:
        print(f"Failed to initialize OC-client for link: {public_share_link}")
        return

    try:
        if not oc.get_file(dataset_fn, bn):
            print(f"Error retrieving file {dataset_fn}. Client failed gracefully")
        else:
            if bn.endswith(".zip"):
                dir_name = bn[:-4]
                !unzip -o $bn -d $dir_name
                return dir_name
            else:
                return bn
    except Exception as e:
      print(f"Error occured while loading/unzipping file {dataset_fn}")
      raise e




def directory_has_files(target_dir):
  n_files = len(glob.glob(os.path.join(target_dir, '*')))
  return n_files > 0


def get_audio_files(cloud_service_type, cloud_link, audio_dir):
  audio_paths = []
  print("Cloud service type", cloud_service_type)

  if cloud_service_type == CLOUD_NEXTCLOUD:
    print(cloud_link)
    cloud_audio_dir = get_data_from_nextcloud(cloud_link, Cloud_File_Name)
    termcolor.cprint(f"Audio directory: {audio_dir}", "green")

    mp3_files = glob.glob(os.path.join(cloud_audio_dir, '*.mp3'))
    wav_files = glob.glob(os.path.join(cloud_audio_dir, '*.wav'))
    audio_paths = mp3_files + wav_files

  elif cloud_service_type == CLOUD_GDRIVE:
    print(cloud_link)
    raise Exception("Not implemented")
  else:
    # See if there are already some audio files
    mp3_files = glob.glob(os.path.join(audio_dir, '*.mp3'))
    wav_files = glob.glob(os.path.join(audio_dir, '*.wav'))
    audio_paths = mp3_files + wav_files

    if len(audio_paths) > 0:
      print(f"Found {len(audio_paths)} files in {audio_dir}, skipping upload.")
      return

    print(f"No files found in {audio_dir}.")
    audio_paths, _ = colab_utils.upload()

  # Copy Audio.
  print('Copying audio to colab for training...')
  for src in audio_paths:
    target = os.path.join(audio_dir,
                          os.path.basename(src).replace(' ', '_'))
    print('Copying {} to {}'.format(src, target))
    shutil.copy(src, target)
    # !cp $src $target


def prepare_dataset(audio_dir,
                    data_dir,
                    sample_rate=16000,
                    frame_rate=50,
                    example_secs=4.0,
                    hop_secs=1.0,
                    viterbi=True,
                    center=True):
  if directory_has_files(data_dir):
    print(f'Dataset already exists in `{data_dir}`')
    return
  else:
    # Otherwise prepare new dataset locally.
    print(f'Preparing new dataset from `{audio_dir}`')

    print()
    print('Creating dataset...')
    print('This usually takes around 2-3 minutes for each minute of audio')
    print('(10 minutes of training audio -> 20-30 minutes)')

    audio_filepattern = os.path.join(audio_dir, '*')
    audio_fp_str = f'"{audio_filepattern}"'
    tfrecord_path_str = f'"{data_dir}/train.tfrecord"'

    !ddsp_prepare_tfrecord \
    --input_audio_filepatterns=$audio_fp_str \
    --output_tfrecord_path=$tfrecord_path_str \
    --num_shards=10 \
    --sample_rate=$sample_rate \
    --frame_rate=$frame_rate \
    --example_secs=$example_secs \
    --hop_secs=$hop_secs \
    --viterbi=$viterbi \
    --center=$center &> /dev/null


def train(model_dir, data_dir, steps=30000, save_steps=300):
  file_pattern = os.path.join(data_dir, 'train.tfrecord*')
  fp_str = f"TFRecordProvider.file_pattern='{file_pattern}'"

  print("-------", file_pattern, "-------")

  # --- CUSTOM ---
  if Threshold:
    # Create a for loop and increase Training_Steps each time
    for i in range(steps // Threshold):
      current_num_steps = (i+1) * Threshold

      print("\n ---Current steps limit:", current_num_steps, "---\n")

      if Threshold_Start <= current_num_steps:
        !ddsp_run \
          --mode=train \
          --save_dir="$model_dir" \
          --gin_file=models/vst/vst.gin \
          --gin_file=datasets/tfrecord.gin \
          --gin_param="$fp_str" \
          --gin_param="TFRecordProvider.centered=True" \
          --gin_param="TFRecordProvider.frame_rate=50" \
          --gin_param="batch_size=16" \
          --gin_param="train_util.train.num_steps=$current_num_steps" \
          --gin_param="train_util.train.steps_per_save=$save_steps" \
          --gin_param="trainers.Trainer.checkpoints_to_keep=3"
  # --- CUSTOM ---

  !ddsp_run \
  --mode=train \
  --save_dir="$model_dir" \
  --gin_file=models/vst/vst.gin \
  --gin_file=datasets/tfrecord.gin \
  --gin_param="$fp_str" \
  --gin_param="TFRecordProvider.centered=True" \
  --gin_param="TFRecordProvider.frame_rate=50" \
  --gin_param="batch_size=16" \
  --gin_param="train_util.train.num_steps=$steps" \
  --gin_param="train_util.train.steps_per_save=$save_steps" \
  --gin_param="trainers.Trainer.checkpoints_to_keep=3"


def reset_state(data_dir, audio_dir, model_dir):
  model_dir_str = f'"{model_dir}"'
  if tf.io.gfile.exists(data_dir):
    !rm -r $data_dir
    !rm -r $audio_dir


def make_dirs(data_dir, audio_dir, model_dir):
  !mkdir -p $data_dir
  !mkdir -p $audio_dir
  !mkdir -p $model_dir


def export_and_download(model_dir, model_name='mymodel'):
  global export_path
  export_path = os.path.join(model_dir, model_name)

  model_dir_str=f'"{model_dir}"'
  export_path_str=f'"{export_path}"'

  !ddsp_export \
  --name=$model_name \
  --model_path=$model_dir_str \
  --save_dir=$export_path_str \
  --inference_model=vst_stateless_predict_controls \
  --tflite \
  --notfjs

  # Zip the whole directory.
  zip_fname = f'{model_name}.zip'
  zip_fp = os.path.join(model_dir, zip_fname)
  print(f'Export complete! Zipping {export_path} to {zip_fp}')
  !cd $model_dir_str && zip -r $zip_fname ./$model_name

  print(f'Zipping Complete! {zip_fname}')
  print(f'You can also find your model at {export_path}')


def get_model_dir(base_dir):
  base_str = 'ddsp-training'
  dirs = tf.io.gfile.glob(os.path.join(base_dir, f'{base_str}-*'))
  if dirs and not Ignore_Previous:
    model_dir = dirs[-1]  # Sorted, so last is most recent.
  else:
    now = datetime.datetime.now().strftime('%Y-%m-%d-%H%M')
    model_dir = os.path.join(base_dir, f'{base_str}-{now}')
  return model_dir


def get_gpu_type():
    gpu_info = []
    try:
        bash_command = "nvidia-smi --query-gpu=name --format=csv"
        output = subprocess.getoutput(bash_command)
        lines = output.split("\n")
        lines.pop(0)
        return lines[0]
    except (OSError, IndexError) as e:
        print("GPU device is not available")
        return ''


def gpu_check():
  gpu_type = get_gpu_type()
  print(f'Using a {gpu_type} GPU...')
  print("Tensorflow Version:", tf.__version__)
  print("Tensorflow all GPUs:", tf.config.list_physical_devices('GPU'))
  print("Tensorflow GPU:", tf.test.gpu_device_name())
  print()


def get_cloud_service_type(cloud_service_string=''):
  cloud_service_type = CLOUD_NONE
  if "cloud" in cloud_service_string.lower():
    cloud_service_type = CLOUD_NEXTCLOUD
    print("Using TUB-Cloud / Nextcloud")

  elif "google" in cloud_service_string.lower():
    cloud_service_type = CLOUD_GDRIVE
    print("Using Google Drive")

  else:
    print('Skipping Cloud Setup...')
    print('Upload Audio Manually...')

  return cloud_service_type

# ------------------------------------------------------------------------------
# Run
# ------------------------------------------------------------------------------
def run(cloud_service='', cloud_link='', base_directory=''):
  cloud_service_type = get_cloud_service_type(cloud_service)

  # Save data locally, but model on drive.
  data_dir = os.path.join(base_directory, 'data')
  audio_dir = os.path.join(base_directory, 'audio')
  model_dir = get_model_dir(base_directory)

  run_training(cloud_service_type=cloud_service_type, cloud_link=cloud_link,
               data_dir=data_dir, audio_dir=audio_dir, model_dir=model_dir)


def run_training(cloud_service_type=0, cloud_link='',
                 data_dir='data', audio_dir='audio', model_dir = 'my_model'):
  # ------------------------------------------------------------------------------
  # Setup
  # ------------------------------------------------------------------------------

  if Ignore_Previous:
    reset_state(data_dir, audio_dir, model_dir)
  make_dirs(data_dir, audio_dir, model_dir)

  # ------------------------------------------------------------------------------
  # Dataset
  # ------------------------------------------------------------------------------
  tick = time.time()

  get_audio_files(cloud_service_type, cloud_link, audio_dir)
  prepare_dataset(audio_dir, data_dir)


  # ------------------------------------------------------------------------------
  # Train
  # ------------------------------------------------------------------------------
  tick = time.time()

  print()
  print('Training...')
  train(model_dir, data_dir, steps=Training_Steps, save_steps=Steps_Per_Save)


  # ------------------------------------------------------------------------------
  # Export
  # ------------------------------------------------------------------------------
  tick = time.time()

  print()
  print('Exporting model...')
  export_and_download(model_dir=model_dir, model_name=Name)

  if Disconnect_Finished:
    print("Dosconnecting from runtime...")
    runtime.unassign()


# The single command.
if Run_Training:
  run(Cloud_Service, Dataset_Link)

In [ ]:
#@title Batch training
#@markdown # Batch training

#@markdown Training of several models in one go over multiple directories/zip-files.
#@markdown Save each result in a seperate directory. <br>
#@markdown Model name will be assumed to be the dir/zip name.

#@markdown Instructions:
#@markdown * Start the cell above ("DDSP-VST Model Training"), with **Run_Training** set to **false**
#@markdown * Provide a link to a dataset below and choose the corresponding cloud service type
#@markdown * Start ▶️ the batch training in the upper left

#@markdown _Note: The link should contain several directories and/or zip-files._

#@markdown <br/>  Link to a Cloud dataset. The link should be public.
Dataset_Link = 'https://tubcloud.tu-berlin.de/s/ye24sGnmtnnN75C'  #@param {type:"string"}

#@markdown <br/> Cloud service type.
Cloud_Service = "TUB-Cloud/Nextcloud" #@param ["TUB-Cloud/Nextcloud"]

# Overwrite global variables
Disconnect_Finished = False
Cloud_File_Name = ''
Ignore_Previous = False  # Would remove the copied audio files

import owncloud
import os


def move_audio_files(src, dst):
  """
  Moves audio files from src to dst, renaming them to ensure they don't have the same filenames.
  No subdirectories will be created.
  """
  mp3_files = glob.glob(os.path.join(src, '**', '*.mp3'), recursive=True)
  wav_files = glob.glob(os.path.join(src, '**', '*.wav'), recursive=True)
  audio_paths = mp3_files + wav_files
  for audio_file in audio_paths:
    base_name, ext = os.path.splitext(os.path.basename(audio_file))
    base_name = base_name.replace(' ', '_')
    destination = os.path.join(dst, f"{base_name}_1{ext}")

    # Ensure the destination file name is unique
    counter = 1
    while os.path.exists(destination):
      counter += 1
      destination = os.path.join(dst, f"{base_name}_{counter}{ext}")

    !mv -f "$audio_file" "$destination"


def run_multiple(cloud_service='', cloud_link='', base_directory='models'):
  # Download content to 'download'
  download_dir = 'download'
  audio_dir = 'audio'
  !mkdir -p $download_dir

  audio_zip_list = []

  oc = owncloud.Client.from_public_link(cloud_link)
  for content_element in oc.list(path=''):
    filename = content_element.get_name()
    local_download_file = (os.path.join(download_dir, filename) + '.zip').replace(' ', '_')

    if content_element.is_dir():
      # get_directory_as_zip OUTDATED!
      # result = oc.get_directory_as_zip(remote_path=content_element.path, local_file=local_download_file[:-4])
      zip_download_link = '"' + cloud_link + '/download?files=' + filename.replace(' ', '%20') + '"'
      !wget $zip_download_link \
         --no-check-certificate --no-proxy --content-disposition \
         -O $local_download_file

    elif filename.lower().endswith('.zip'):
      local_download_file = local_download_file[:-4]
      oc.get_file(remote_path=filename, local_file=local_download_file)

    audio_zip_list.append(local_download_file)

  print(f"Running training for {len(audio_zip_list)} models...")
  tmp_dir = 'tmp'

  # Create a sperate directory for each content of 'download'
  for i, audio_zip in enumerate(audio_zip_list):
    print("---------- Model", i+1, "----------")
    dirmodelname = os.path.join(base_directory, os.path.basename(audio_zip)[:-4])
    dirmodelaudio = os.path.join(dirmodelname, audio_dir)
    print(dirmodelaudio)

    # Empty model audio directory
    !rm -r $dirmodelaudio

    # Create necessary directories
    !mkdir -p $dirmodelname
    !mkdir -p $dirmodelaudio
    !mkdir -p $tmp_dir

    # Unzip to tmp
    !unzip -o -d $tmp_dir $audio_zip

    # Move .wav and .mp3 files to 'audio'
    #!find $tmp_dir -type f \( -iname \*.wav -o -iname \*.mp3 \) -exec sh -c 'cp "{}" "$dirmodelaudio/$(basename "{}" | sed "s/\.[^.]*$//;s/$/_1.wav/")"' \;
    #!find $tmp_dir -type f \( -iname \*.wav -o -iname \*.mp3 \) -exec sh -c 'cp "{}" "$dirmodelaudio/$(basename "{}" | sed "s/\.[^.]*$//;s/$/_1.wav/")"' \;
    move_audio_files(tmp_dir, dirmodelaudio)

    # Remove the tmp directory for later models
    !rm -r $tmp_dir

    # Set the name of the model (name of the model directory)
    global Name
    Name = os.path.basename(dirmodelname)

    # Run training and set 'base_directory' every time
    run(cloud_service='', cloud_link='', base_directory=dirmodelname)


run_multiple(cloud_service=Cloud_Service, cloud_link=Dataset_Link)

In [ ]:
#@title  Loss visualizaion
#@markdown #  Loss visualizaion
#@markdown Run this code after the training is finished to visualize the loss over time. </br>
#@markdown If a batch training was used, just the last result will be displayed.
%load_ext tensorboard
#%reload_ext tensorboard

#@markdown <br>Leave empty to use summaries location from the last training above.
Summaries_Location = '/content/ddsp-training-2024-01-16-1530/summaries/train' #@param {type:"string"}
#export_path = '/content/gdrive/MyDrive/AnVoGen/Cat/ddsp-training-2024-01-10-1302/cat'

logs = Summaries_Location
if Summaries_Location is '':
  MODEL_DIR = os.path.dirname(export_path)
  logs = os.path.join(MODEL_DIR, 'summaries')

print("Loading logs from:", logs)

%tensorboard --logdir $logs